# Exploration 05_ STT        
---
## 이 노드의 루브릭      
1. 음성데이터를 2차원 Spectogram으로 변환하여 데이터셋을 구성하였다.      
    -> "스펙토그램 시각화 및 train/test 데이터셋 구성이 정상 진행되었다."    
    
    
2. 1, 2차원 데이터를 처리하는 음성인식 모델이 정상 작동한다.          
    -> "스펙토그램을 입력받은 모델이 학습 과정에서 안정적으로 수렴하며, evaluation/test 단계를 무리없이 진행 가능하다.      
    
    
    
3. 테스트셋 수행 결과 음성인식 모델의 Accuracy가 일정 수준에 도달하였다.     
    -> "평가 결과 75% 이상의 정확도를 달성하는 모델이 하나 이상 존재한다."    

---
### 목차       

1. 데이터 처리와 분류    
  - 라벨 데이터 처리하기
  - sklearn의 train_test_split 함수를 이용하여 train, test 분리   
2. 학습을 위한 하이퍼파라미터 설정     
3. 데이터셋 구성        
  - tf.data.Dataset을 이용       
  - from_tensor_slices에 return 받길 원하는 데이터를 튜플 형태로 받음   
  - map과 batch를 사용한 데이터 전처리.
  - 메모리 비우기 수행하기    
  
  > del speech_data
  > del spec_data
  
4. 2차원 Spectogram 데이터를 처리하는 모델 구성     
  - 2차원 Spectogram 데이터의 시간축 방향으로 Conv1D/Conv2D 레이어를 적용가능    
  - batchnorm, dropout, dense layer 를 이용    
  - 12개의 단어 class를 구분하는 loss를 사용하고, Adam optimizer 사용   
  - 모델 가중치를 저장하는 checkpoint callback 함수 추가     
  
5. 학습 내용 그래프 출력
  - loss, accuracy를 그래프로 표현     
  
6. Test dataset을 이용한 모델의 성능 평가   
  - 저장한 weight 불러오기    
  - 모델의 예측값과 정답값이 얼마나 맞는지 확인

### 1. 데이터 처리와 분류      
##### 데이터 가져오기       
* 기존에는 Waveform 데이터로 입력받아, Text라벨을 출력하는 모델을 만들었다.    
* 이제 이 데이터를 Spectogram으로 입력받아, 동일 역할을 수행하는 모델을 만든다.

In [35]:
import numpy as np
import os

data_path = os.getenv("HOME")+'/SUBMIT_MISSION_GIT/ex5_STT/data/speech_wav_8000.npz'
speech_data = np.load(data_path)

print("데이터를 가져왔다!")
print("wav 상태 그대로의 데이터 셰이프 :", speech_data["wav_vals"].shape)

데이터를 가져왔다!
wav 상태 그대로의 데이터 셰이프 : (50620, 8000)


##### 내겐 너무 어려웠던 데이터 형변환         
* 사실, speech_data -> spectogram data로 형변환하는 과정이 너무 어려웠다.   
* 하나하나씩 차근히 분해해보기로 했다.         
---
1. 일단, speech_data를 이루고 있는 저 npz 확장자는 대체 무엇인지     
* npz 파일 포맷은 여러 개의 리스트를 한번에 저장하기 위한 포맷이다.     
* 그렇군.      
---
* 그럼, 얘 안에 들어있는 데이터들은 뭘까?      
* 위에서 speech_data를 살펴보았을 때, 데이터는     
* speech_data["wav_vals"]와 speech_data["label_vals"]가 있었다.    
* wav_vals는 음성 데이터, label_vals는 음성의 뜻을 의미하는 라벨 데이터이다.  
* 프린트해서 다시 확인해보자.

In [3]:
#wav_vals와 label_vals 인쇄
print("음성 데이터 shape : " ,speech_data["wav_vals"].shape)
print("\n어떻게 생겼나?" , speech_data["wav_vals"][0])

print("\n라벨 데이터 shape : " ,speech_data["label_vals"].shape)
print("\n어떻게 생겼나?" , speech_data["label_vals"][0])

음성 데이터 shape :  (50620, 8000)

어떻게 생겼나? [-1.27418665e-04 -1.12644804e-04 -1.86756923e-04 ... -1.62762426e-05
 -4.93293861e-04 -3.55132594e-04]

라벨 데이터 shape :  (50620, 1)

어떻게 생겼나? ['down']


* 인쇄해봤더니, 위와 같았다.     
* 음성데이터는 (음성개수:50620개, 샘플링 레이트 8000)       
* 라벨데이터는 (라벨개수:50620개, 라벨 값이 들어간 1개)   
* 의 모양으로 생긴 것을 확인할 수 있다.       
---
__그럼 우리는 이제 spectogram 데이터가 어떤 모양인지 알아야 한다!__   

![데이터 변환](./PostingPic/data.png)         

* 사진을 보면, 노드에서 wav데이터를 spectogram으로 변환할 때     
* Waveform shape(8000,) 스펙토그램(130,126)으로 바뀐 것을 알 수 있다.    
* 그럼 일렬로 된 8000개의 샘플링 데이터를 -> 음성 1개당 (130,126)의 데이터로 변환시켜야 한다. 

In [36]:
#wav 데이터를 spectogram으로 변환한다.
import librosa

#wav 데이터를 spectogram 데이터로 바꾸는 매직
#fft_size는 스펙토그램 사이즈를 맞추기 위해 258로 고정   
def wav2spec(wav, fft_size=258):
    D = np.abs(librosa.stft(wav, n_fft=fft_size))
    return D

#변환한 data를 담아줄 리스트
specData =[]

for wav_data in speech_data["wav_vals"]:
    specData.append(wav2spec(wav_data))
    
print(specData[0])

[[8.0723902e-03 5.2032182e-03 3.4693219e-03 ... 1.8922662e-02
  3.3809434e-04 1.0142580e-02]
 [6.5486785e-03 3.2852096e-03 2.2495915e-03 ... 2.5527487e-02
  1.6525777e-02 3.2326202e-03]
 [2.4226354e-03 3.5080472e-03 2.4726372e-03 ... 2.7952474e-02
  2.6693283e-02 1.7628349e-02]
 ...
 [5.8292970e-04 2.9035914e-04 1.4131786e-05 ... 6.0245253e-05
  3.1307511e-04 5.0933618e-04]
 [5.2434229e-04 2.5860392e-04 1.2995474e-05 ... 2.1275569e-05
  2.2292971e-04 4.7651122e-04]
 [5.1448052e-04 2.5908535e-04 1.6500426e-06 ... 5.2643327e-06
  2.3299157e-04 4.3895244e-04]]


* 리스트로 만들었으니, 이 리스트의 형태를 연산할 수 있게 np.array로 변환해야 한다.

In [ ]:
specData = np.array(specData)

print("Spectrogram shape : ",specData.shape)
print("1개 데이터의 모양 : ", specData[0].shape)

* 50620개의 데이터를 1데이터 당 (130,126) 사이즈로 변환했다!

### 1. 데이터 처리하기 - 1) 라벨 데이터 처리하기          

* 그럼 이제 라벨 데이터도 같은 모양을 갖도록 처리해보자!     
* 현재의 라벨 데이터 모양을 출력해본다.       

In [11]:
print(speech_data["label_vals"].shape)

(50620, 1)


* 라벨 데이터의 종류들을 확인해보자.

In [15]:
label_list = np.unique(speech_data["label_vals"])
print(label_list)

['down' 'go' 'left' 'no' 'off' 'on' 'right' 'silence' 'stop' 'unknown'
 'up' 'yes']


* 이 12개의 데이터들을, 학습에 사용하기 위해 딕셔너리 형태로 바꿔준다.    
* 단, 현재 0:down, 1:go.... 로 되어있는 리스트를 거꾸로 down:0 이 되도록 바꿔주어야 컴퓨터가 처리 가능하다.   

In [16]:
label_data_dict = dict() 

for i, l in enumerate(label_list):
    label_data_dict[l] = i
    
label_list = label_data_dict

print("indexed labels : " , label_data_dict)

indexed labels :  {'down': 0, 'go': 1, 'left': 2, 'no': 3, 'off': 4, 'on': 5, 'right': 6, 'silence': 7, 'stop': 8, 'unknown': 9, 'up': 10, 'yes': 11}


* 12개의 라벨이 모두 __라벨(str값):인덱스  의 형태로 변환__ 된 것을 확인할 수 있다.     
* 여기서 다시 라벨을 연산이 가능한 np.array 형태로 바꿔준다.

In [21]:
temp = []

for value in speech_data["label_vals"]:
    temp.append(label_list[value[0]])

print("전부 다 변환됐는가? : " , len(temp))
label_data = np.array(temp)

print(label_data.shape)
print("몇 개만 출력해보자 " ,  label_data[9] , label_data[2850])

전부 다 변환됐는가? :  50620
(50620,)
몇 개만 출력해보자  0 1


* 확인해보니, 50620개의 데이터가 모두 label_data 형태로 변환되었음을 알 수 있다. 

### 1.데이터 처리하기 - 2. 학습 데이터, 테스트 데이터 분리

In [33]:
from sklearn.model_selection import train_test_split

#차원 알려주기
sp_s=130
sp_p=126

# 학습데이터:테스트데이터 = 9:1 로 해 보았다 ㅎㅎ
train_spec, test_spec, train_label, test_label = train_test_split(specData, label_data, test_size=0.1, shuffle=True)
print("학습 데이터 수", len(train_spec))

train_spec = train_spec.reshape([-1, sp_s, sp_p,1])
test_spec = test_spec.reshape([-1, sp_s, sp_p,1])

#나눠진 데이터 모습을 확인해보자.
print("train_spec : ", train_spec.shape)
print("train_label :" , train_label.shape)
print("test_spec : ", test_spec.shape)
print("test_label :", test_label.shape)

NameError: name 'specData' is not defined

### 2. 학습을 위한 하이퍼파라미터 설정          

* 혹시 모르는 마음에, 기본적인 설정은 노드와 동일하게 진행해주었다.      
* 추후 성능에 따라 하이퍼파라미터를 조정해주도록 한다.    

In [27]:
batch_size = 32
max_epoch = 10

#중간중간 저장해줄 체크포인트 생성
check_path = os.getenv('HOME')+'/SUBMIT_MISSION_GIT/ex5_STT/model/wav'

check_path

'/home/ssac23/SUBMIT_MISSION_GIT/ex5_STT/model/wav'

### 3. 데이터셋 구성 - 1) 데이터셋 구성     

---
__요건__       
* tf.data.Dataset을 이용        
* from_tensor_slices에 return 받길 원하는 데이터를 튜플 형태로 받음     
* map과 batch를 사용한 데이터 전처리

In [28]:
#정답 라벨 말고는 다 죽이는 one_hot_label
def one_hot_label(spec,label):
    
    #depth=12는 최종 결과값이 12개 나온다는 의미(물론 뒤에 1차원이 더 붙는다.)
    #우리 라벨 데이터는 12개이므로 depth=12임!
    label = tf.one_hot(label, depth=12)
    return spec, label

print("one_hot_label")

one_hot_label


In [29]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((train_spec, train_label))
train_dataset = train_dataset.map(one_hot_label)
train_dataset = train_dataset.repeat().batch(batch_size=batch_size)
print(train_dataset)

test_dataset = tf.data.Dataset.from_tensor_slices((test_spec, test_label))
test_dataset = test_dataset.map(one_hot_label)
test_dataset = test_dataset.batch(batch_size=batch_size)
print(test_dataset)

print("데이터 처리 완료")

<BatchDataset shapes: ((None, 130, 126), (None, 12)), types: (tf.float32, tf.float32)>
<BatchDataset shapes: ((None, 130, 126), (None, 12)), types: (tf.float32, tf.float32)>
데이터 처리 완료


__메모리 비우기 수행하기__

In [ ]:
del speech_data
del specData
print("비우기 완료!")

### 4. 2차원 Spectogram 데이터를 처리하는 모델 구성      
---   
__요건__     
* 2차원 spectogram 데이터의 시간축 방향으로 Conv1D, Conv2D 레이어를 적용 가능   
* batchnorm, dropout, dense layer를 이용    
* 12개의 단어 class를 구분하는 loss를 사용하고, Adam optimizer 사용     
* 모델 가중치를 저장하는 checkpoint callback 함수 추가

In [32]:
from tensorflow.keras import layers

#spec_data의 형태를 알려준다.
input_tensor = layers.Input(shape=train_spec.shape)

x = layers.Conv1D(32, 9, padding='same', activation='relu')(input_tensor)
x = layers.Conv1D(32, 9, padding='same', activation='relu')(x)
x = layers.MaxPool1D()(x)

x = layers.Conv1D(64, 9, padding='same', activation='relu')(x)
x = layers.Conv1D(64, 9, padding='same', activation='relu')(x)
x = layers.MaxPool1D()(x)

x = layers.Conv1D(128, 9, padding='same', activation='relu')(x)
x = layers.Conv1D(128, 9, padding='same', activation='relu')(x)
x = layers.Conv1D(128, 9, padding='same', activation='relu')(x)
x = layers.MaxPool1D()(x)

x = layers.Conv1D(256, 9, padding='same', activation='relu')(x)
x = layers.Conv1D(256, 9, padding='same', activation='relu')(x)
x = layers.Conv1D(256, 9, padding='same', activation='relu')(x)
x = layers.MaxPool1D()(x)
x = layers.Dropout(0.3)(x)

x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)

output_tensor = layers.Dense(12)(x)

model_spec = tf.keras.Model(input_tensor, output_tensor)

model_spec.summary()

ValueError: Input 0 of layer conv1d is incompatible with the layer: expected ndim=3, found ndim=4. Full shape received: [None, 45558, 130, 126]